In [11]:
import pandas as pd
import random
import numpy as np


SEED = 42
random.seed(42)
np.random.seed(SEED)

X = pd.read_csv("train_processed.csv", index_col=None)
X = X.drop("Name", axis=1)
y = X.pop("Transported")

test_X = pd.read_csv("test_processed.csv", index_col=None)
test_X = test_X.drop("Name", axis=1)

test_X

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinDeck,CabinNum,CabinSide,GroupId,MemberId
0,0,0,0,27.0,0,0.0,0.0,0.0,0.0,0.0,0,3,0,13,1
1,0,1,0,19.0,0,0.0,9.0,0.0,2823.0,0.0,1,4,0,18,1
2,1,0,1,31.0,0,0.0,0.0,0.0,0.0,0.0,2,0,0,19,1
3,1,1,0,38.0,0,0.0,6652.0,0.0,181.0,585.0,2,1,0,21,1
4,0,1,0,20.0,0,10.0,0.0,635.0,0.0,0.0,1,5,0,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0,0,0,34.0,0,0.0,0.0,0.0,0.0,0.0,0,1496,0,9266,2
4273,0,1,0,42.0,0,0.0,847.0,17.0,10.0,144.0,0,160,1,9269,1
4274,2,0,1,18.0,0,0.0,0.0,0.0,0.0,0.0,4,296,1,9271,1
4275,1,1,0,18.0,0,0.0,2680.0,0.0,0.0,523.0,4,297,1,9273,1


In [12]:
X = X[["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "RoomService"]]
test_X = test_X[["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "RoomService"]]
X

,Age,FoodCourt,ShoppingMall,Spa,VRDeck,RoomService
0,39.0,0.0,0.0,0.0,0.0,0.0
1,24.0,9.0,25.0,549.0,44.0,109.0
2,58.0,3576.0,0.0,6715.0,49.0,43.0
3,33.0,1283.0,371.0,3329.0,193.0,0.0
4,16.0,70.0,151.0,565.0,2.0,303.0
...,...,...,...,...,...,...
8688,41.0,6819.0,0.0,1643.0,74.0,0.0
8689,18.0,0.0,0.0,0.0,0.0,0.0
8690,26.0,0.0,1872.0,1.0,0.0,0.0
8691,32.0,1049.0,0.0,353.0,3235.0,0.0


In [13]:
from sklearn.model_selection import train_test_split

train_X, dev_X, train_y, dev_y = train_test_split(X, y, train_size=0.8, random_state=42)
len(train_X), len(dev_X)

(6954, 1739)

# Dev

In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score

clf = AdaBoostClassifier()
clf.fit(train_X, train_y)
dev_preds = clf.predict(dev_X)

f1_score(dev_y, dev_preds)

0.796422935297212

# Submission

In [15]:
clf = AdaBoostClassifier()
clf.fit(X, y)
preds = clf.predict(test_X)
preds

array([1, 0, 1, ..., 1, 1, 1])

In [16]:
test_Ids = pd.read_csv("test.csv")["PassengerId"]
test_data = [[idx, bool(pred)] for idx, pred in zip(test_Ids, preds)]
submission_df = pd.DataFrame(data=test_data, columns=["PassengerId", "Transported"])
submission_df

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [17]:
submission_df.to_csv("submission.csv", index=False)

In [18]:
# ! kaggle competitions submit -c spaceship-titanic -f submission.csv -m "limited features, AdaBoostClassifier"

100%|██████████████████████████████████████| 56.1k/56.1k [00:00<00:00, 88.3kB/s]
Successfully submitted to Spaceship Titanic